In [8]:
import pandas as pd

file_path = '../Data/240515 Subject 5 labelled.xlsx'
df = pd.read_excel(file_path)
df_breath = df[df["Label phase 1"] == "Breathing"]
breathing_indexes = df_breath.index
print(df_breath.head())

        Time Label phase 1 Breathing Cadence Label Phase 2
18  00:00:38     Breathing                 x     Breathing
19  00:00:40     Breathing                 x     Breathing
20  00:00:42     Breathing                 x     Breathing
21  00:00:44     Breathing                 x     Breathing
22  00:00:46     Breathing                 x     Breathing


In [7]:
breathing_indexes = df_breath.index